In [1]:
! python --version

Python 3.12.0


In [2]:
%%capture
! pip install llama-index
! pip install psycopg2-binary
! pip install SQLAlchemy
! pip install python-dotenv
! pip install llama-index-llms-bedrock
! pip install llama-index-embeddings-bedrock

In [3]:
from sqlalchemy import (
    create_engine,
)
import os

from dotenv import load_dotenv

load_dotenv(verbose=True, dotenv_path="../../../.env")

engine = create_engine(f"postgresql+psycopg2://postgres:{
                       os.environ["PG_VECTOR_PW"]}@localhost:5432/{os.environ["PG_VECTOR_DB"]}")

In [4]:
# Testing our connection
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM people LIMIT 5")
    print(cursor.fetchall())
    
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM planet LIMIT 5")
    print(cursor.fetchall())

[(1, 'Luke Skywalker', 172, 77.0, 'blond', 'fair', 'blue', '19BBY', 'male', 1, datetime.datetime(2014, 12, 9, 13, 50, 51, 644000), datetime.datetime(2014, 12, 20, 21, 17, 56, 891000), 'https://swapi.co/api/people/1/'), (2, 'C-3PO', 167, 75.0, 'n/a', 'gold', 'yellow', '112BBY', 'n/a', 1, datetime.datetime(2014, 12, 10, 15, 10, 51, 357000), datetime.datetime(2014, 12, 20, 21, 17, 50, 309000), 'https://swapi.co/api/people/2/'), (3, 'R2-D2', 96, 32.0, 'n/a', 'white, blue', 'red', '33BBY', 'n/a', 8, datetime.datetime(2014, 12, 10, 15, 11, 50, 376000), datetime.datetime(2014, 12, 20, 21, 17, 50, 311000), 'https://swapi.co/api/people/3/'), (4, 'Darth Vader', 202, 136.0, 'none', 'white', 'yellow', '41.9BBY', 'male', 1, datetime.datetime(2014, 12, 10, 15, 18, 20, 704000), datetime.datetime(2014, 12, 20, 21, 17, 50, 313000), 'https://swapi.co/api/people/4/'), (5, 'Leia Organa', 150, 49.0, 'brown', 'light', 'brown', '19BBY', 'female', 2, datetime.datetime(2014, 12, 10, 15, 20, 9, 791000), datetim

In [5]:
from llama_index.core import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding

Settings.llm = Bedrock(
    # model="anthropic.claude-3-haiku-20240307-v1:0",
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"],
    region_name=os.environ["AWS_DEFAULT_REGION"],
)

Settings.embed_model = BedrockEmbedding(
    model_name="cohere.embed-english-v3",
    region_name=os.environ["AWS_DEFAULT_REGION"],
)

In [6]:
from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine

all_table_names = ["people", "planet"]

sql_database = SQLDatabase(engine, include_tables=all_table_names)

In [7]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display

In [8]:
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []
for table_name in all_table_names:
    table_schema_objs.append(SQLTableSchema(table_name=table_name))
    
print(table_schema_objs)

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=2)
)

def format_sql_results_as_markdown_table(sql_results, headers):
    """
    Formats the SQL results as a markdown table
    """
    if not sql_results and not headers:
        return "No results found."
    
    header_row = "| " + " | ".join(headers) + " |"
    separator_row = "| " + " | ".join(["---"] * len(headers)) + " |"

    table_rows = []
    for row in sql_results:
        table_row = "|"
        for j, _ in enumerate(headers):
            table_row += f" {str(row[j])} |"
        table_rows.append(table_row)

    markdown_table = "\n".join([header_row, separator_row] + table_rows)
    return markdown_table

response_template = """
### Question

**{number}. {question}**

### Answer

{response}

### Generated SQL Query
```sql
{sql}
```

### SQL Results

{sql_results}

"""

def text_to_sql(query_engine, question, number=1):
    """
    Calls the query engine with the given question and displays the response as a markdown cell
    """
    engine_response = query_engine.query(question)
    display(Markdown(response_template.format(
            number=number,
            question=question,
            response=str(engine_response),
            sql=engine_response.metadata["sql_query"],
            sql_results=format_sql_results_as_markdown_table(engine_response.metadata["result"], engine_response.metadata["col_keys"]),
        )))

[SQLTableSchema(table_name='people', context_str=None), SQLTableSchema(table_name='planet', context_str=None)]


In [9]:
text_to_sql(query_engine, "How many tables are there in the database?", number=1)


### Question

**1. How many tables are there in the database?**

### Answer

Based on the SQL query and response, there are 2 tables in the database schema 'public'.

### Generated SQL Query
```sql
SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public';
```

### SQL Results

| count |
| --- |
| 2 |



In [10]:
text_to_sql(query_engine, "How many of the planets are rocky?", number=2)


### Question

**2. How many of the planets are rocky?**

### Answer

Based on the SQL query and result, there are 20 planets in the database that have either "rocky" or "mountains" in their terrain description. Therefore, the response to the input question "How many of the planets are rocky?" would be:

There are 20 rocky or mountainous planets.

### Generated SQL Query
```sql
SELECT COUNT(*) 
FROM planet
WHERE terrain LIKE '%rocky%' OR terrain LIKE '%mountains%';
```

### SQL Results

| count |
| --- |
| 20 |



In [11]:
text_to_sql(query_engine, "Give me the names, hair color of people who have blue eyes? Arrange the names in reverse alphabetical order.", number=3)


### Question

**3. Give me the names, hair color of people who have blue eyes? Arrange the names in reverse alphabetical order.**

### Answer

Here are the names and hair colors of people with blue eyes, arranged in reverse alphabetical order:

Wilhuff Tarkin - auburn, grey hair
Tarfful - brown hair  
Ric Olié - brown hair
Qui-Gon Jinn - brown hair
Owen Lars - brown, grey hair
Mon Mothma - auburn hair
Mas Amedda - no hair
Luminara Unduli - black hair
Luke Skywalker - blond hair
Lobot - no hair
Jocasta Nu - white hair
Jek Tono Porkins - brown hair
Finis Valorum - blond hair  
Cliegg Lars - brown hair
Chewbacca - brown hair
Beru Whitesun lars - brown hair
Barriss Offee - black hair
Anakin Skywalker - blond hair
Adi Gallia - no hair

### Generated SQL Query
```sql
SELECT name, hair_color 
FROM people
WHERE eye_color = 'blue'
ORDER BY name DESC;
```

### SQL Results

| name | hair_color |
| --- | --- |
| Wilhuff Tarkin | auburn, grey |
| Tarfful | brown |
| Ric Olié | brown |
| Qui-Gon Jinn | brown |
| Owen Lars | brown, grey |
| Mon Mothma | auburn |
| Mas Amedda | none |
| Luminara Unduli | black |
| Luke Skywalker | blond |
| Lobot | none |
| Jocasta Nu | white |
| Jek Tono Porkins | brown |
| Finis Valorum | blond |
| Cliegg Lars | brown |
| Chewbacca | brown |
| Beru Whitesun lars | brown |
| Barriss Offee | black |
| Anakin Skywalker | blond |
| Adi Gallia | none |



In [12]:
text_to_sql(query_engine, "List the top 10 planets that have rocky characteristics, in alphabetical order?", number=4)


### Question

**4. List the top 10 planets that have rocky characteristics, in alphabetical order?**

### Answer

Based on the SQL query and results, here is a possible response to the input question "List the top 10 planets that have rocky characteristics, in alphabetical order?":

The top 10 planets with rocky characteristics, listed in alphabetical order, are:

1. Alderaan - Featuring grasslands and mountains
2. Bestine IV - With rocky islands and oceans
3. Cato Neimoidia - Comprised of mountains, fields, forests, and rock arches
4. Coruscant - A cityscape planet with mountains
5. Dantooine - Containing oceans, savannas, mountains, and grasslands
6. Endor - Known for its forests, mountains, and lakes
7. Geonosis - A rocky, desert planet with mountains and barren landscapes
8. Hoth - An icy tundra world with ice caves and mountain ranges
9. Iktotch - Described as rocky
10. Iridonia - Featuring rocky canyons and acid pools

### Generated SQL Query
```sql
SELECT p.name, p.terrain
FROM planet p
WHERE p.terrain LIKE '%rocky%' OR p.terrain LIKE '%mountain%'
ORDER BY p.name
LIMIT 10;
```

### SQL Results

| name | terrain |
| --- | --- |
| Alderaan | grasslands, mountains |
| Bestine IV | rocky islands, oceans |
| Cato Neimoidia | mountains, fields, forests, rock arches |
| Coruscant | cityscape, mountains |
| Dantooine | oceans, savannas, mountains, grasslands |
| Endor | forests, mountains, lakes |
| Geonosis | rock, desert, mountain, barren |
| Hoth | tundra, ice caves, mountain ranges |
| Iktotch | rocky |
| Iridonia | rocky canyons, acid pools |



In [13]:
text_to_sql(query_engine, "List the top 10 planets that have rocky characteristics, who are the men and describe their skin color and eye color?", number=5)


### Question

**5. List the top 10 planets that have rocky characteristics, who are the men and describe their skin color and eye color?**

### Answer

Based on the SQL query and results, here is a synthesized response to the input question:

The top 10 planets with rocky characteristics and the details of the men from those planets are:

1. Bestine IV - Jek Tono Porkins has fair skin and blue eyes.
2. Iktotch - Saesee Tiin has pale skin and orange eyes. 
3. Iridonia - Eeth Koth has brown skin and brown eyes.
4. Sullust - Nien Nunb has grey skin and black eyes.

The query filtered for planets with 'rocky' in their terrain description, joined with the people table to get the male residents' details like name, skin color, and eye color. It ordered the results by planet name and limited the output to 10 rows.

### Generated SQL Query
```sql
SELECT p.name, pe.name, pe.skin_color, pe.eye_color
FROM planet p
JOIN people pe ON p.id = pe.planet_id
WHERE p.terrain LIKE '%rocky%' AND pe.gender = 'male'
ORDER BY p.name
LIMIT 10;
```

### SQL Results

| name | name | skin_color | eye_color |
| --- | --- | --- | --- |
| Bestine IV | Jek Tono Porkins | fair | blue |
| Iktotch | Saesee Tiin | pale | orange |
| Iridonia | Eeth Koth | brown | brown |
| Sullust | Nien Nunb | grey | black |



In [23]:
text_to_sql(query_engine, "What are the names and heights of all people who are from planets with a temperate climate? Please only list the top 10.", number=6)


### Question

**6. What are the names and heights of all people who are from planets with a temperate climate? Please only list the top 10.**

### Answer

Here are the names and heights of the top 10 people from planets with a temperate climate:

1. Lama Su - 229 cm
2. Roos Tarpals - 224 cm  
3. Grievous - 216 cm
4. Taun We - 213 cm
5. Rugor Nass - 206 cm
6. Tion Medon - 206 cm
7. Ki-Adi-Mundi - 198 cm
8. Jar Jar Binks - 196 cm  
9. Kit Fisto - 196 cm
10. Mas Amedda - 196 cm

### Generated SQL Query
```sql
SELECT p.name, p.height
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.climate LIKE '%temperate%'
ORDER BY p.height DESC
LIMIT 10;
```

### SQL Results

| name | height |
| --- | --- |
| Lama Su | 229 |
| Roos Tarpals | 224 |
| Grievous | 216 |
| Taun We | 213 |
| Rugor Nass | 206 |
| Tion Medon | 206 |
| Ki-Adi-Mundi | 198 |
| Jar Jar Binks | 196 |
| Kit Fisto | 196 |
| Mas Amedda | 196 |



In [24]:
text_to_sql(query_engine, "What are the names and birth years of all people from planets with a population greater than 1,000,000? Also include the polulation. Top 10 only please.", number=7)


### Question

**7. What are the names and birth years of all people from planets with a population greater than 1,000,000? Also include the polulation. Top 10 only please.**

### Answer

Here is a response synthesized from the query results:

The top 10 people from planets with a population greater than 1,000,000 are:

1. Jocasta Nu from a planet with a population of 1,000,000,000,000
2. Finis Valorum, born in 91BBY, from a planet with a population of 1,000,000,000,000  
3. Adi Gallia from a planet with a population of 1,000,000,000,000
4. Wat Tambor from a planet with a population of 500,000,000,000
5. Poggle the Lesser from a planet with a population of 100,000,000,000
6. Ackbar, born in 41BBY, from a planet with a population of 27,000,000,000
7. Wilhuff Tarkin, born in 64BBY, from a planet with a population of 22,000,000,000
8. Nien Nunb from a planet with a population of 18,500,000,000
9. Jabba Desilijic Tiure, born in 600BBY, from a planet with a population of 7,000,000,000
10. San Hill from a planet with a population of 5,000,000,000

### Generated SQL Query
```sql
SELECT p.name, p.birth_year, pl.population
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE pl.population > 1000000
ORDER BY pl.population DESC
LIMIT 10;
```

### SQL Results

| name | birth_year | population |
| --- | --- | --- |
| Jocasta Nu | None | 1000000000000 |
| Finis Valorum | 91BBY | 1000000000000 |
| Adi Gallia | None | 1000000000000 |
| Wat Tambor | None | 500000000000 |
| Poggle the Lesser | None | 100000000000 |
| Ackbar | 41BBY | 27000000000 |
| Wilhuff Tarkin | 64BBY | 22000000000 |
| Nien Nunb | None | 18500000000 |
| Jabba Desilijic Tiure | 600BBY | 7000000000 |
| San Hill | None | 5000000000 |



In [16]:
text_to_sql(query_engine, "What is the average rotation period of all planets?", number=8)


### Question

**8. What is the average rotation period of all planets?**

### Answer

The average rotation period of all planets is approximately 26 days.

### Generated SQL Query
```sql
SELECT AVG(rotation_period) AS average_rotation_period
FROM planet;
```

### SQL Results

| average_rotation_period |
| --- |
| 25.9583333333333333 |



In [17]:
text_to_sql(query_engine, "List the names, birthyear, and URLs of all people born before the year 20BBY.", number=9)


### Question

**9. List the names, birthyear, and URLs of all people born before the year 20BBY.**

### Answer

Here is a response synthesized from the query results:

The people born before the year 20BBY include Dooku (born 102BBY), C-3PO (born 112BBY), IG-88 (born 15BBY), Luke Skywalker (born 19BBY), Leia Organa (born 19BBY), and Chewbacca (born 200BBY). Dooku and C-3PO were born over a century before the Battle of Yavin, while IG-88, Luke, and Leia were born in the decades just prior. Chewbacca, a wookiee from the planet Kashyyyk, was born an astonishing 200 years before the events of the original Star Wars trilogy. You can find more information about each of these characters at the URLs provided in the query results.

### Generated SQL Query
```sql
SELECT name, birth_year, url 
          FROM people
          WHERE birth_year < '20BBY'
          ORDER BY birth_year;
```

### SQL Results

| name | birth_year | url |
| --- | --- | --- |
| Dooku | 102BBY | https://swapi.co/api/people/67/ |
| C-3PO | 112BBY | https://swapi.co/api/people/2/ |
| IG-88 | 15BBY | https://swapi.co/api/people/23/ |
| Luke Skywalker | 19BBY | https://swapi.co/api/people/1/ |
| Leia Organa | 19BBY | https://swapi.co/api/people/5/ |
| Chewbacca | 200BBY | https://swapi.co/api/people/13/ |



In [18]:
text_to_sql(query_engine, "List the names of all female people and their planet's name.", number=10)


### Question

**10. List the names of all female people and their planet's name.**

### Answer

Based on the SQL query and results, here is a possible response:

The query lists the names of all female people and the name of the planet they are from. Some notable entries include:

- Leia Organa from Alderaan
- Padmé Amidala from Naboo  
- Rey from an unknown planet
- Shaak Ti from Shili
- Shmi Skywalker from Tatooine
- Jocasta Nu from Coruscant
- Luminara Unduli from Mirial
- Adi Gallia from Coruscant
- Ayla Secura from Ryloth
- Mon Mothma from Chandrila

The results show a diverse range of female characters from many different planets across the Star Wars universe. Some are Jedi like Adi Gallia and Luminara Unduli, while others are leaders like Padmé Amidala and Mon Mothma. The list also includes droids like R4-P17 whose gender is listed as female.

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [19]:
# Now query in German (List the names of all female people and their planet's name.)
text_to_sql(query_engine, "Liste die Namen aller weiblichen Personen und den Namen ihres Planeten auf.", number=11)


### Question

**1. Liste die Namen aller weiblichen Personen und den Namen ihres Planeten auf.**

### Answer

Die Abfrage listet die Namen aller weiblichen Personen und den Namen ihrer jeweiligen Heimatplaneten auf. Hier sind die Ergebnisse:

Adi Gallia vom Planeten Coruscant
Ayla Secura vom Planeten Ryloth  
Barriss Offee vom Planeten Mirial
Beru Whitesun lars vom Planeten Tatooine
Captain Phasma von einem unbekannten Planeten
Cordé vom Planeten Naboo
Dormé vom Planeten Naboo
Jocasta Nu vom Planeten Coruscant
Leia Organa vom Planeten Alderaan
Luminara Unduli vom Planeten Mirial
Mon Mothma vom Planeten Chandrila
Padmé Amidala vom Planeten Naboo
R4-P17 von einem unbekannten Planeten
Rey von einem unbekannten Planeten
Shaak Ti vom Planeten Shili
Shmi Skywalker vom Planeten Tatooine
Sly Moore vom Planeten Umbara
Taun We vom Planeten Kamino
Zam Wesell vom Planeten Zolan

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [20]:
# Now query in Korean (List the names of all female people and their planet's name.)
text_to_sql(query_engine, "모든 여성의 이름과 그들의 행성 이름을 나열하세요.", number=12)


### Question

**12. 모든 여성의 이름과 그들의 행성 이름을 나열하세요.**

### Answer

여성 이름과 행성 이름은 다음과 같습니다:

Adi Gallia - Coruscant
Ayla Secura - Ryloth  
Barriss Offee - Mirial
Beru Whitesun lars - Tatooine
Captain Phasma - 알 수 없음
Cordé - Naboo
Dormé - Naboo
Jocasta Nu - Coruscant
Leia Organa - Alderaan
Luminara Unduli - Mirial
Mon Mothma - Chandrila
Padmé Amidala - Naboo
R4-P17 - 알 수 없음
Rey - 알 수 없음
Shaak Ti - Shili
Shmi Skywalker - Tatooine
Sly Moore - Umbara
Taun We - Kamino
Zam Wesell - Zolan

### Generated SQL Query
```sql
SELECT p.name AS person_name, pl.name AS planet_name
FROM people p
JOIN planet pl ON p.planet_id = pl.id
WHERE p.gender = 'female'
ORDER BY p.name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |



In [21]:
# Now query in Filipino (List the names of all female people and their planet's name.)
text_to_sql(query_engine, "Isalista ang mga pangalan ng lahat ng babae at ang pangalan ng kanilang planeta.", number=13)


### Question

**13. Isalista ang mga pangalan ng lahat ng babae at ang pangalan ng kanilang planeta.**

### Answer

Narito ang mga pangalan ng lahat ng babae at ang kanilang planeta:

Adi Gallia - Coruscant
Ayla Secura - Ryloth  
Barriss Offee - Mirial
Beru Whitesun lars - Tatooine
Captain Phasma - Unknown
Cordé - Naboo
Dormé - Naboo
Jocasta Nu - Coruscant
Leia Organa - Alderaan
Luminara Unduli - Mirial
Mon Mothma - Chandrila
Padmé Amidala - Naboo
R4-P17 - Unknown
Rey - Unknown
Shaak Ti - Shili
Shmi Skywalker - Tatooine
Sly Moore - Umbara
Taun We - Kamino
Zam Wesell - Zolan

### Generated SQL Query
```sql
SELECT people.name AS person_name, planet.name AS planet_name
FROM people 
JOIN planet ON people.planet_id = planet.id
WHERE people.gender = 'female'
ORDER BY person_name;
```

### SQL Results

| person_name | planet_name |
| --- | --- |
| Adi Gallia | Coruscant |
| Ayla Secura | Ryloth |
| Barriss Offee | Mirial |
| Beru Whitesun lars | Tatooine |
| Captain Phasma | Unknown |
| Cordé | Naboo |
| Dormé | Naboo |
| Jocasta Nu | Coruscant |
| Leia Organa | Alderaan |
| Luminara Unduli | Mirial |
| Mon Mothma | Chandrila |
| Padmé Amidala | Naboo |
| R4-P17 | Unknown |
| Rey | Unknown |
| Shaak Ti | Shili |
| Shmi Skywalker | Tatooine |
| Sly Moore | Umbara |
| Taun We | Kamino |
| Zam Wesell | Zolan |

